In [ ]:
from numpy import exp
from numpy.fft import fft2,  fftfreq, fftshift, ifftshift, rfftfreq
from SFS.src.py.utils import *

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150

In [ ]:
number = 6
folder = "data/SETD_paper/{number}/".format(number = number)
time = get_time(folder + "1/")
n = count_files(folder)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,5))

rr = []
Cs = []

# Number to skip
skip = 1
# number to average over
av = 300

indx = range(0, n, skip)
for i in indx:
    q0, Cq, con, T, L, N = get_Cq_saved(folder, seed=i+1)
    assert av <= np.shape(Cq)[0]
    Cq = np.mean(np.abs(Cq[-av:None, :]), axis=0)
    
    r = con["r"]
    rr.append(r)
    color = cm.viridis(i/(n-1))
    title = "$r = {r:.3f}$".format(r=r)
    
    ax[0].loglog(q0[1:], Cq[1:, 0, 0], lw=1., ls="-", color=color, zorder=0)
    x = Cq[1, 0, 0]
    Cs.append(x)
    p = ax[1].plot(r, x, 'x', color=color)


q0, Cq, con, T, L, N = get_Cq_saved(folder, seed=1)
u = con["u"]
ax[0].plot(q0[1:], T*q0[1:]**(-2), 'k--', label="$\\frac{T}{q^2}$")
ax[0].plot(q0[1:], 1.45*T*q0[1:]**(-1.75), 'k--', label="$\\frac{T}{q^{2-\\nu}}$")

title = "$u = {u}, N = {N}, L = {L}, T={T}, t={t:1.0e} $".format(u=u,N=N,L=L, T=T,t=time[-1])

fig.colorbar(cm.ScalarMappable(norm=colors.Normalize(vmin=np.min(rr), vmax=np.max(rr)), cmap=cm.viridis), ax=ax, label="$r$")
fig.suptitle(title, y=1.05)
ax[0].legend()
ax[0].set_ylabel("$C(q)$")
ax[0].set_xlabel("$q$")
ax[1].set_ylabel("$C(0)$")
ax[1].set_xlabel("$a$")

plt.show()

# Correlation Length

In [ ]:
from numpy import log as ln
from scipy.optimize import curve_fit

s, e = 1, 25

fig, ax = plt.subplots(figsize=(6,4))

xi = []
rs = []
As = []
Bs = []

q0, Cq, con, T, L, N = get_Cq_saved(folder, seed=1)
ax.plot(q0[s:e], T*q0[s:e]**(-2), 'k--', label="$\\frac{T}{q^2}$")
ax.plot(q0[s:e], 1.45*T*q0[s:e]**(-1.75), 'k--', label="$\\frac{T}{q^{2-\\nu}}$")

for j in indx:

    q0, Cq, con, T, L, N = get_Cq_saved(folder, seed=j+1)
    assert av <= np.shape(Cq)[0]
    Cq = np.mean(np.abs(Cq[-av:None, :]), axis=0)[:, 0, 0]
    Cq, q0 = Cq[(Cq!=0)], q0[(Cq!=0)]
    Cq, q0 = Cq[s:e], q0[s:e]
    r = con["r"]

    f = lambda q, A, B, xi : A / (1 + (xi * q)**B )
    lnf = lambda q, A, B, xi : np.log(A / (1 + (q * xi)**B))
    
    XI = 20
    p0 = (T*XI**2, 2, XI)
    
    color = cm.viridis(j/(n-1)) 
    ax.loglog(q0, Cq, 'o', ms=3, color=color)
    ax.loglog(q0, f(q0, *p0), color='black', lw=1.)
    try: 
        # p = curve_fit(f, q0, Cq, p0=p0)
        p = curve_fit(lnf, q0, np.log(Cq),p0=p0)

        rs.append(r)
        xi.append(p[0][2])
        As.append(p[0][0])
        Bs.append(p[0][1])
        ax.loglog(q0, f(q0, *p[0]), color=color, lw=1.)

    except: 
        print("Could not fit {i}".format(i=j))
        continue


title = "$u = {u}, N = {N}, L = {L}, T={T}, t={t:1.0e} $".format(u=u,N=N,L=L, T=T,t=time[-1])
print(title)

ax.set_ylabel("$C(q)$")
ax.set_xlabel("$q$")

fig.colorbar(cm.ScalarMappable(norm=colors.Normalize(vmin=np.min(rs), vmax=np.max(rs)), cmap=cm.viridis), ax=ax, label="$r$",)

rs, xi, As, Bs = rs[::-1], xi[::-1], As[::-1], Bs[::-1]
rs, xi, As, Bs = np.array(rs), np.array(xi), np.array(As), np.array(Bs)

In [ ]:
xi_c = np.argmax(xi)
B_c = np.argmin(Bs)
A_c = np.argmax(As)

ic = xi_c
rc  = rs[ic]
print(rc)

nn = 0
mm = xi_c

fig, ax = plt.subplots(1, 1, figsize=(6,5)) 
fig.suptitle("$C(q; r) = A(r) \{1 + [\\xi(r) q]^{ B(r) }\}^{-1}$")
ax.plot(rs, Bs, ".-", color="g", label="$B(r)$", lw=2, ms=8)

style1 = dict(label="$\\xi_c$", color="blue", marker="x", linestyle="", ms=10, zorder=10)
style2 = dict(label="$A_c$", color="purple", marker="$\circ$", linestyle="", ms=10, zorder=11)
style3 = dict(label="$B_c$",color="orange", marker="+",  linestyle="", ms=12, zorder=12)


ax.plot(rs[xi_c], Bs[xi_c], **style1)
ax.plot(rs[A_c], Bs[A_c], **style2)
ax.plot(rs[B_c], Bs[B_c], **style3)

ax.set_ylabel("$B$")
ax.set_xlabel("$r$")

ax.legend()
plt.show()


fig, ax = plt.subplots(1, 1, figsize=(6,5)) 
ax2 = ax.twinx()
ax.plot(rs, As, 'k-', marker=".", lw=2, ms=8, label = "$A$", zorder=1)
ax2.plot(rs, xi, 'r--', marker=".", lw=2, ms=8, label="$\\xi$", zorder=1)

ax2.plot(rs[xi_c], xi[xi_c],**style1)
ax2.plot(rs[A_c], xi[A_c],  **style2)
ax2.plot(rs[B_c], xi[B_c],  **style3)

ax.set_ylabel("$A$")
ax2.set_ylabel("$\\xi$")
ax.set_xlabel("$r$")
 

lines2, labels2 = ax.get_legend_handles_labels()
lines1, labels1 = ax2.get_legend_handles_labels()

ax.legend(lines2+lines1, labels2+labels1, loc=1)

plt.show()

In [ ]:
# # Plot all (including orange)
pa      = 1
alpha   = 0.7
ms      = 6
f       = lambda X, a, b : a * X + b

def plot_xy(ax1, ax2, x, y, rc=None, pa=pa, mm=mm+1, nn=nn, color='k', sgn=1):
    if mm==0: mm = None
    xx, yy = x[nn:mm], y[nn:mm]
    p = curve_fit(f, xx, yy)[0]
    label = "${p1:.2f}x + c$".format(p1=p[0],p2=p[1])

    if pa: ax1.loglog(exp(x), exp(y), 'o', ms=ms, color="orange")
    ax1.loglog(exp(xx), exp(f(xx, *p)), 'k--', lw=5, label=label, alpha=alpha, zorder=0)    
    ax1.loglog(exp(xx), exp(yy), 'o', ms=ms, color="tab:blue")

    if  rc is None: rc = 0 
    else: ax2.plot([rc, rc], [0, np.max(exp(y))], '--', color=color)
    if pa: ax2.plot(sgn*exp(x) + rc, exp(y), '.', color='orange')

    ax2.plot(sgn*exp(xx) + rc, exp(f(xx, *p)), '--', color=color)
    ax2.plot(sgn*exp(xx) + rc , exp(yy), '.', color="tab:blue")

In [ ]:
# Plot all
pa      = 0
alpha   = 0.7
ms      = 6
lw      = 2
f       = lambda X, a, b : a * X + b

def plot_xy(ax1, ax2, x, y, rc=None, pa=pa, mm=mm+1, nn=nn, color='k', sgn=1):
    if mm==0: mm = None
    xx, yy = x[nn:mm], y[nn:mm]
    p = curve_fit(f, xx, yy)[0]
    label = "${p1:.2f}x + c$".format(p1=p[0],p2=p[1])

    if pa: 
        ax1.loglog(exp(x), exp(y), 'o', ms=ms, color="orange")
        ax1.loglog(exp(x), exp(f(x, *p)), 'k--', lw=lw, label=label, alpha=alpha, zorder=10)
    else:
        ax1.loglog(exp(xx), exp(f(xx, *p)), 'k--', lw=lw, label=label, alpha=alpha, zorder=10)
    ax1.loglog(exp(xx), exp(yy), 'o', ms=ms, color="tab:blue")

    if  rc is None: rc = 0 
    else: ax2.plot([rc, rc], [np.min(exp(y)), np.max(exp(y))], '--', color=color)

    ax2.plot(sgn*exp(xx) + rc , exp(yy), '.', color="tab:blue", zorder=3)

    a, b = exp(np.min(yy)), exp(np.max(yy))
    if pa: 
        ax2.plot(sgn*exp(x) + rc, exp(y), '.', color='orange', zorder=2)
        a, b = exp(np.min(y)), exp(np.max(y))
        ax2.plot(sgn*exp(x) + rc, exp(f(x, *p)), '--', color=color, zorder=1)
    else:
        ax2.plot(sgn*exp(xx) + rc, exp(f(xx, *p)), '--', color=color, zorder=1)
    

    d = (b-a)/20
    ax2.set_ylim(a-d, b+d)

In [ ]:
fig0, ax0 = plt.subplots(1, 3, figsize=(15, 5))
fig1, ax1 = plt.subplots(1, 3, figsize=(15, 5))
fig2, ax2 = plt.subplots(1, 3, figsize=(15, 5))

title = "$u = {u}, N = {N}, L = {L}, T={T}, t={t:1.0e}, r_c={rc:1.4f} $".format(u=u,N=N,L=L, T=T,t=time[-1], rc=rc)
fig0.suptitle(title)
fig1.suptitle(title)

# Plot all
pa = 1

# Above critical point

# Critical index
ic = xi_c + 4
rc = rs[ic]
print(rc)

# Points to exlude close to crit
mm = xi_c - 1
# Points to exlude far from crit
nn = 0
#

lnr = ln(np.abs(rs-rc)[:ic])
lnxi = ln(xi[:ic])
lnA = ln(As[:ic])

plot_xy(ax0[0], ax2[0], lnr, lnxi, rc=rc, mm=mm, nn=nn, pa=pa)
plot_xy(ax0[1], ax2[1], lnr, lnA, rc=rc, mm=mm, nn=nn, pa=pa)
plot_xy(ax0[2], ax2[2], lnxi, lnA, mm=mm, nn=nn, pa=pa)


# Below critical point
ic = xi_c
rc  = rs[ic]
# Points to exlude close to crit
nn = 0
# Points to exlude far from crit
mm = xi_c

lnr = ln(np.abs(rs-rc)[ic+1:])
lnxi = ln(xi[ic+1:])
lnA = ln(As[ic+1:])

plot_xy(ax1[0], ax2[0], lnr, lnxi, rc=rc, nn=nn, mm=mm, color="r", sgn=-1, pa=pa)
plot_xy(ax1[1], ax2[1], lnr, lnA, rc=rc, nn=nn, mm=mm, color="r", sgn=-1, pa=pa)
plot_xy(ax1[2], ax2[2], lnxi, lnA, rc=rc, nn=nn, mm=mm, color="r", pa=pa)

for ax in [ax0, ax1]:
    ax[0].set_xlabel("$|r - r_c|$")
    ax[0].set_ylabel("$\\xi$")
    ax[1].set_xlabel("$|r - r_c|$")
    ax[1].set_ylabel("$A_0$")
    ax[2].set_xlabel("$\\xi$")
    ax[2].set_ylabel("$A_0$")
 
for ax in np.concatenate([ax0, ax1]):
    ax.xaxis.set_minor_formatter(plt.NullFormatter())
    ax.yaxis.set_minor_formatter(plt.NullFormatter())
    ax.legend()

fig0.tight_layout(rect=[0, 0, 1, 1.05]) 
fig1.tight_layout(rect=[0, 0, 1, 1.05]) 
fig2.tight_layout(rect=[0, 0, 1, 1.05]) 
fig0.savefig("fig/figexp.pdf")
